In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
import pandas as pd
import parse
from model import *

In [3]:
# replay = os.path.join( "train", os.listdir("train")[0])
# data = parse.parse_replay_file(replay)

data = parse.parse_replay_folder("train", "teccles")

Load Replay: train\ts2018-halite-3-gold-replays_replay-20181020-192338+0000-1540063416-40-40-764395.hlt
Load Basic Information
Load Cell Information
Load Player Ships
Load Other Player Ships
Load Droppoff Information
Load Replay: train\ts2018-halite-3-gold-replays_replay-20181020-192454+0000-1540063480-48-48-764423.hlt
Load Basic Information
Load Cell Information
Load Player Ships
Load Other Player Ships
Load Droppoff Information
Load Replay: train\ts2018-halite-3-gold-replays_replay-20181020-201720+0000-1540066636-48-48-765185.hlt
Load Basic Information
Load Cell Information
Load Player Ships
Load Other Player Ships
Load Droppoff Information
Load Replay: train\ts2018-halite-3-gold-replays_replay-20181020-201908+0000-1540066745-40-40-765218.hlt
Load Basic Information
Load Cell Information
Load Player Ships
Load Other Player Ships
Load Droppoff Information
Load Replay: train\ts2018-halite-3-gold-replays_replay-20181020-202134+0000-1540066858-64-64-765263.hlt
Load Basic Information
Load 

In [4]:
game_states = []
for player_data in data:
    game_states.append(HaliteModel.process_game_states(player_data))
        
game_states = np.array(game_states)

Processing Game States
Processing Game States
Processing Game States
Processing Game States
Processing Game States
Processing Game States
Processing Game States
Processing Game States
Processing Game States
Processing Game States
Processing Game States
Processing Game States
Processing Game States
Processing Game States
Processing Game States
Processing Game States
Processing Game States
Processing Game States
Processing Game States
Processing Game States


In [5]:
game_states.shape

(20,)

In [7]:
m = HaliteModel(data=data)
features, labels = [], []

for i, game in enumerate(game_states):
    print(f"Game {i+1}/{len(game_states)}")
    f, l = m.generate_training_data(game)
    
    features.extend(f)
    labels.extend(l)
    print(i)
    

Game 1/20
Generating Training Data



0
Game 2/20
Generating Training Data



1
Game 3/20
Generating Training Data



2
Game 4/20
Generating Training Data



3
Game 5/20
Generating Training Data


...


15
Game 17/20
Generating Training Data



16
Game 18/20
Generating Training Data



17
Game 19/20
Generating Training Data



18
Game 20/20
Generating Training Data



19


In [8]:
features = np.array(features)
labels = np.array(labels)

features.shape, labels.shape

((213176, 78), (213176,))

In [9]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [10]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.65)

No noticeable improvement found when regularizing.

In [12]:
model = MLPClassifier(solver="adam", hidden_layer_sizes=(256, 128, 64, 32), max_iter=1000, learning_rate_init=1e-5)
model.fit(X_train, y_train)

C:\ProgramData\Anaconda\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(256, 128, 64, 32), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [13]:
preds = model.predict(X_test)
accuracy_score(preds, y_test)

0.6148869016307207

In [42]:
import pickle
with open("deep3.nn", 'wb') as f:
    pickle.dump(model, f)

In [15]:
pd.DataFrame({"y":y_train}).y.value_counts()

0    12925
2     1964
4     1913
3     1873
1     1797
Name: y, dtype: int64

In [11]:
from sklearn.utils import resample

https://elitedatascience.com/imbalanced-classes

In [12]:
df = None
features.shape

(213176, 78)

In [13]:
df = pd.DataFrame(X_train) # pd.merge([pd.DataFrame(features), pd.Series(labels.T)])
df["y"] = y_train
print(df.shape)
df.y.value_counts()

(74611, 79)


0    45202
3     7421
4     7360
2     7350
1     7278
Name: y, dtype: int64

In [14]:
majority = df[df.y == 0]
minority = df[df.y != 0]

min_upsampled = resample(minority, replace=True, n_samples=len(majority)*2)

upsampled = pd.concat([majority.sample(len(majority)//2), min_upsampled])
upsampled.y.value_counts()

3    22918
4    22763
0    22601
2    22386
1    22337
Name: y, dtype: int64

In [15]:
upsampled_y = upsampled.y
upsampled_X = upsampled.drop("y", axis=1)


dfa = pd.DataFrame(X_test) # pd.merge([pd.DataFrame(features), pd.Series(labels.T)])
dfa["y"] = y_test
print(dfa.shape)
dfa.y.value_counts()


(138565, 79)


0    84006
1    13714
2    13642
4    13632
3    13571
Name: y, dtype: int64

In [67]:
model = MLPClassifier(solver="adam", hidden_layer_sizes=(100, 50), max_iter=1000, learning_rate_init=1e-3)

model.fit(upsampled_X, upsampled_y)
model.score(X_test, y_test)

0.6240283145872237

In [72]:

x = [5, 10, 50 ,75,100,250, 500, 1000]
for i in x:
    print(i)
    model = MLPClassifier(solver="adam", hidden_layer_sizes=(i, i), max_iter=1000, learning_rate_init=1e-3)

    model.fit(upsampled_X, upsampled_y)
    import pickle
    with open("width"+str(i)+".nn", 'wb') as f:
        pickle.dump(model, f)
    print(model.score(X_test, y_test))

5
0.5921454524398894
10
0.6054993413529045
50
0.6105079544303065
75
0.6330539511587846
100
0.6514454046698803
250
0.6685629912711165
500
0.6732458997408839
1000
0.6675714016878737


In [ ]:
pd.Series(model.predict(X_test)).value_counts()

In [ ]:
import pickle
with open("deep3.nn", 'wb') as f:
    pickle.dump(model, f)

In [19]:
x = [1,2,5,10,25]
for i in x:
    print(i)
    model = MLPClassifier(solver="adam", hidden_layer_sizes=(100)*i, max_iter=1000, learning_rate_init=1e-3)

    model.fit(upsampled_X, upsampled_y)
    import pickle
    with open("depth"+str(i)+".nn", 'wb') as f:
        pickle.dump(model, f)
    print(model.score(X_test, y_test))

1
0.6008443690686681
2
0.5983329123516039
5
0.5939667304153285
10
0.5941543679861437
25
0.5901345938729117
